In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder, StandardScaler
import torch
pd.set_option('display.max_columns', None)

In [2]:
dataPath = "data/"

pairs = pd.read_csv(dataPath + "relation.csv")
users = pd.read_csv(dataPath + "customers.csv", index_col="id")
items = pd.read_csv(dataPath + "products.csv", index_col="id")


/tmp/ipykernel_832/1774659839.py:4: DtypeWarning: Columns (7,9,10,11,12,13,17,18,21,26,27,28,30,36) have mixed types. Specify dtype option on import or set low_memory=False.
  users = pd.read_csv(dataPath + "customers.csv", index_col="id")
/tmp/ipykernel_832/1774659839.py:5: DtypeWarning: Columns (20,32,34,39) have mixed types. Specify dtype option on import or set low_memory=False.
  items = pd.read_csv(dataPath + "products.csv", index_col="id")


### Compañia 15

In [3]:
df_pairs = pairs[ pairs.company_id == 15]#.copy()
df_users = users[ users.company_id == 15]#.copy()
df_items = items[ items.company_id == 15]#.copy()

In [4]:
# Comprobacion de que no haya usuarios ni productos duplicados

assert df_users.index.duplicated().any()==False
assert df_items.index.duplicated().any()==False

### Quitar usuarios que interaccionan pero no estan contemplados en la tabla usuarios (unos 575)

In [5]:
usuarios_que_interaccionan_pero_no_en_tabla_usuarios = set(df_pairs.customer_id) - set(df_users.index)
len(usuarios_que_interaccionan_pero_no_en_tabla_usuarios)

575

In [6]:
df_pairs = df_pairs[ ~df_pairs.customer_id.isin(usuarios_que_interaccionan_pero_no_en_tabla_usuarios) ]
len(df_pairs)

59167

### Quitar productos que interaccionan pero no estan contemplados en la tabla productos (unos 162)

In [7]:
productos_que_interaccionan_pero_no_en_tabla_productos = set(df_pairs.product_id) - set(df_items.index)
len(productos_que_interaccionan_pero_no_en_tabla_productos)

162

In [8]:
df_pairs = df_pairs[ ~df_pairs.product_id.isin(productos_que_interaccionan_pero_no_en_tabla_productos) ]
len(df_pairs)

58913

# Contar usuarios y priductos

In [9]:
ACTIVE_USERS = df_pairs.customer_id.unique()
ACTIVE_ITEMS = df_pairs.product_id.unique()

real2fake_userId = {userId:i for i,userId in enumerate(ACTIVE_USERS)}
real2fake_itemId = {itemId:i for i,itemId in enumerate(ACTIVE_ITEMS)}

NUM_ACTIVE_USERS = len(ACTIVE_USERS)
NUM_ACTIVE_ITEMS = len(ACTIVE_ITEMS)

NUM_USERS = len(df_users)
NUM_ITEMS = len(df_items)

print(f"Hy un total de {NUM_ACTIVE_USERS} usuarios que interaccionan de un total de {NUM_USERS} usuarios")
print(f"Hy un total de {NUM_ACTIVE_ITEMS} prudcutos que interaccionan de un total de {NUM_ITEMS} productos")

Hy un total de 28806 usuarios que interaccionan de un total de 54029 usuarios
Hy un total de 477 prudcutos que interaccionan de un total de 780 productos


## Extract feats

In [10]:
df_pairs.sample(2)

,company_id,product_id,customer_id,compras,visto,peso,ultimo
88466,15,29143,56779,1,0,3.0,2021-03-17 15:28:16
59189,15,28923,154945,1,0,3.0,2022-01-24 20:21:23


### Extract user features: gender (4)

In [11]:
df_user_feats = pd.get_dummies(df_users.gender_id, dummy_na = True).astype("float32")  # Categorica, 1-hot encoding (4)
df_user_feats

,3.0,4.0,5.0,NaN
id,,,,
21724,0.0,0.0,1.0,0.0
21725,0.0,0.0,1.0,0.0
21726,0.0,0.0,1.0,0.0
21727,0.0,0.0,1.0,0.0
21728,0.0,0.0,1.0,0.0
...,...,...,...,...
537633,0.0,0.0,1.0,0.0
537635,1.0,0.0,0.0,0.0
537637,0.0,1.0,0.0,0.0


### Extract item features: price (1) + families (37) + features (1027) = 1065

In [12]:
df_item_feats = pd.concat(
    [
        (( df_items.price - df_items.price.mean() ) / df_items.price.std() ).to_frame(), # Continua (1)
        df_items.families.str.get_dummies(','), # Categorica, N-hot encoding (37)
        df_items.features.str.get_dummies(','), # Categorica, N-hot encoding (1027)
    ],
    axis="columns"    
).astype("float32")

df_item_feats

price  1462  1463  1464  1465  1466  1467  1468  1469  1471  1472  \
id                                                                             
28863  -0.172383   1.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
28864   0.009349   1.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
28865   0.099989   1.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
28866   0.235948   1.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
28867   0.598506   1.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
...          ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   
126891 -0.624675   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
126951 -0.624675   0.0   0.0   0.0   0.0   0.0   0.0   0.0   1.0   0.0   0.0   
126952 -0.624675   0.0   0.0   0.0   0.0   0.0   0.0   0.0   1.0   0.0   0.0   
126957 -0.624675   1.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
126958 -0.624675   0.0   0.0   0.0   0.0   0.0   0.0   0.0   1.0   0.0   0.0   

        1473  1474  1475  1476  1477  1478  1479  1480  1481  1482  1484  \
id                                                                         
28863    0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
28864    0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
28865    0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
28866    0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
28867    0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
...      ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   
126891   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
126951   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
126952   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
126957   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
126958   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   

        1486  1488  1490  1491  1492  1493  1494  1495  1496  1497  1500  \
id                                                                         
28863    0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
28864    0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
28865    0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
28866    0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
28867    0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
...      ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   ...   
126891   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
126951   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
126952   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
126957   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   
126958   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   

        1503  1544  1675  2627  3492  10040  10185  10191  10206  10239  \
id                                                                        
28863    0.0   0.0   0.0   0.0   0.0    0.0    0.0    0.0    0.0    0.0   
28864    0.0   0.0   0.0   0.0   0.0    0.0    0.0    0.0    0.0    0.0   
28865    0.0   0.0   0.0   0.0   0.0    0.0    0.0    0.0    0.0    0.0   
28866    0.0   0.0   0.0   0.0   0.0    0.0    0.0    0.0    0.0    0.0   
28867    0.0   0.0   0.0   0.0   0.0    0.0    0.0    0.0    0.0    0.0   
...      ...   ...   ...   ...   ...    ...    ...    ...    ...    ...   
126891   0.0   0.0   0.0   0.0   0.0    0.0    0.0    0.0    0.0    0.0   
126951   0.0   0.0   0.0   0.0   0.0    0.0    0.0    0.0    0.0    0.0   
126952   0.0   0.0   0.0   0.0   0.0    0.0    0.0    0.0    0.0    0.0   
126957   0.0   0.0   0.0   0.0   0.0    0.0    0.0    0.0    0.0    0.0   
126958   0.0   0.0   0.0   0.0   0.0    0.0    0.0    0.0    0.0    0.0   

        10289  1

In [13]:
class CollabFilteringDataset(torch.utils.data.Dataset):
    
    def __init__(self, df_pairs, df_user_feats, df_item_feats):
        
        self.df_pairs = df_pairs
        self.df_pairs["fake_user_id"] = self.df_pairs["customer_id"].map(real2fake_userId)
        self.df_pairs["fake_item_id"] = self.df_pairs["product_id"].map(real2fake_itemId)
        
        self.df_user_feats = df_user_feats
        self.df_item_feats = df_item_feats
        

    def __len__(self):
        return len(self.df_pairs)

    def __getitem__(self, idx):
        
        pairs_row = self.df_pairs.iloc[idx]
        
        item_fake_id  = pairs_row.fake_item_id # Para usarlo en el embedding
        item_id       = pairs_row.product_id   # ... para obetener mas datos del item (clave ajena)
        
        user_fake_id  = pairs_row.fake_item_id # Para usarlo en el embedding
        user_id       = pairs_row.customer_id  # ...para obetener mas datos del user (clave ajena)
        
        peso          = pairs_row.peso
                
        return {"Item_id": item_fake_id, 
                "User_id": user_fake_id,
                "Item_feats": self.df_item_feats.loc[item_id].values,
                "User_feats": self.df_user_feats.loc[user_id].values,
                "Peso": peso.astype("float32")}

In [14]:
ds = CollabFilteringDataset(df_pairs, df_user_feats, df_item_feats)

In [15]:
ds[0]

{'Item_id': 0,
 'User_id': 0,
 'Item_feats': array([-0.17238298,  1.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ], dtype=float32),
 'User_feats': array([1., 0., 0., 0.], dtype=float32),
 'Peso': 3.0}

In [16]:
train_size = int(0.8 * len(ds))
valid_size = len(ds) - train_size
train_ds, valid_ds = torch.utils.data.random_split(ds, [train_size, valid_size])

train_dl = torch.utils.data.DataLoader(train_ds, batch_size=64, shuffle=True)
valid_dl = torch.utils.data.DataLoader(valid_ds, batch_size=64, shuffle=False)

## Model

In [17]:
class Model(torch.nn.Module):
    
    def __init__(self): # n_users, n_movies, n_factors, y_range=(0,5.5)
        
        super().__init__()
        
        COMMON_EMB_DIM = 32
        
        self.user_id_embed_W = torch.nn.Embedding(NUM_ACTIVE_USERS, COMMON_EMB_DIM)
        self.user_id_embed_B = torch.nn.Embedding(NUM_ACTIVE_USERS, 1)
        self.user_linear_1   = torch.nn.Linear(4, COMMON_EMB_DIM) # 4 (genero)
                
        self.item_id_embed_W = torch.nn.Embedding(NUM_ACTIVE_ITEMS, COMMON_EMB_DIM)
        self.item_id_embed_B = torch.nn.Embedding(NUM_ACTIVE_ITEMS, 1)
        self.item_linear_1   = torch.nn.Linear(1065, 200) # 1 (precio) + 37 (familias) + 1027 (features)
        self.item_linear_2   = torch.nn.Linear(200, COMMON_EMB_DIM)
        
        """
        self.user_factors = Embedding(n_users, n_factors)
        self.user_bias    = Embedding(n_users, 1)
        self.movie_factors = Embedding(n_movies, n_factors)
        self.movie_bias = Embedding(n_movies, 1)
        self.y_range = y_range
        """;
        
    def encode_user(self, real_user_id, user_feats):        
        
        user_emb_from_feats = self.user_linear_1(user_feats)
        
        try:
            # Este usuario ya ha interaccionado
            # Su embedding sera por sus atributos y por su id
            fake_userId = real2fake_userId[real_user_id]
            user_emb_from_id = self.user_id_embed_W(fake_userId)
            return user_emb_from_feats + user_emb_from_id
        
        except KeyError:
            # Este usuario todavía no ha interaccionado
            # Su embedding sera solo por sus atributos
            return user_emb_from_feats
        
    def encode_item(self, real_item_id, item_feats):        
        
        item_emb_from_feats = self.item_linear_1(item_feats)
        item_emb_from_feats = torch.nn.functional.relu(item_emb_from_feats)
        item_emb_from_feats = self.item_linear_2(item_emb_from_feats)
        
        try:
            # Este usuario ya ha interaccionado
            # Su embedding sera por sus atributos y por su id
            fake_itemId = real2fake_itemId[real_item_id]
            item_emb_from_id = self.item_id_embed_W(fake_itemId)
            return item_emb_from_feats + item_emb_from_id
        
        except KeyError:
            # Este usuario todavía no ha interaccionado
            # Su embedding sera solo por sus atributos
            return item_emb_from_feats
    
    
    def encode_active_user(self, user_id, user_feats):        
        user_emb_from_id    = self.user_id_embed_W(user_id)
        user_emb_from_feats = self.user_linear_1(user_feats)
        
        return user_emb_from_feats + user_emb_from_id
        
        
    def encode_active_item(self,  item_id, item_feats):
        item_emb_from_id    = self.item_id_embed_W(item_id)
        item_emb_from_feats = self.item_linear_1(item_feats)
        item_emb_from_feats = torch.nn.functional.relu(item_emb_from_feats)
        item_emb_from_feats = self.item_linear_2(item_emb_from_feats)
        
        return item_emb_from_feats + item_emb_from_id

    
    def forward(self, x):
        
        user_embedding = self.encode_active_user(x["User_id"], x["User_feats"])
        item_embedding = self.encode_active_item(x["Item_id"], x["Item_feats"])
        
        return (user_embedding * item_embedding).sum(axis=1)

## Train

In [18]:
learning_rate = 0.001

model = Model()
loss  = torch.nn.MSELoss()
optim = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [24]:
error = -1
for epoch in range(5):
    for batch in tqdm(train_dl):
        optim.zero_grad()
        model_out = model(batch)
        error = loss(model_out, batch["Peso"])
        error.backward()
        optim.step()
    print(error)

100%|████████████████████████████████| 737/737 [00:12<00:00, 60.08it/s]


tensor(1.3413, grad_fn=<MseLossBackward0>)


100%|████████████████████████████████| 737/737 [00:11<00:00, 61.82it/s]


tensor(1.9302, grad_fn=<MseLossBackward0>)


100%|████████████████████████████████| 737/737 [00:11<00:00, 61.95it/s]


tensor(1.1664, grad_fn=<MseLossBackward0>)


100%|████████████████████████████████| 737/737 [00:11<00:00, 61.65it/s]


tensor(1.8992, grad_fn=<MseLossBackward0>)


100%|████████████████████████████████| 737/737 [00:11<00:00, 61.60it/s]

tensor(1.9824, grad_fn=<MseLossBackward0>)


## Create embs

In [24]:
user_ids_np   = df_user_feats.index.values
user_feats_np = df_user_feats.to_numpy()

user_ids_tensor   = torch.from_numpy(user_ids_np)
user_feats_tensor = torch.from_numpy(user_feats_np)

user_embs = model.encode_user( user_ids_tensor, user_feats_tensor )


user_ids_tensor.shape, user_feats_tensor.shape, user_embs.shape

(torch.Size([54029]), torch.Size([54029, 4]), torch.Size([54029, 32]))

In [25]:
item_ids_np   = df_item_feats.index.values
item_feats_np = df_item_feats.to_numpy()

item_ids_tensor   = torch.from_numpy(item_ids_np)
item_feats_tensor = torch.from_numpy(item_feats_np)

item_embs = model.encode_item( item_ids_tensor, item_feats_tensor )

item_ids_tensor.shape, item_feats_tensor.shape, item_embs.shape

(torch.Size([780]), torch.Size([780, 1065]), torch.Size([780, 32]))